<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## 1. Obtain data from Wikipedia and create dataframe

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import requests

Scraping Wikipedia table with BeautifulSoup library

In [2]:
#Extract the data
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_doc = requests.get(page).text
soup = BeautifulSoup(html_doc, 'html.parser')

#Transform the data into a pandas dataframe
column_names = ['PostalCode','Borough', 'Neighborhood'] 
nbh = pd.DataFrame(columns=column_names)

In [3]:
#Loop through the data and fill the dataframe
for link in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = link.find_all(['th','td'])
    try:
        PostalCode = data[0].text
        Borough = data[1].text
        Neighbourhood = data[2].text
        
        if (Borough == 'Not assigned'):
            continue
        else:
            nbh = nbh.append({'PostalCode': PostalCode,
                                    'Borough': Borough,
                                    'Neighborhood': Neighbourhood},ignore_index=True)   
    except IndexError:pass

In [4]:
nbh['Neighborhood'] = nbh['Neighborhood'].str.replace('\n', '')
nbh['Neighborhood'] = nbh['Neighborhood'].replace('Not assigned', np.nan)
nbh = nbh.dropna(axis=0, subset=['Neighborhood'])
nbh.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern
10,M3B,North York,Don Mills North


Group the data

In [5]:
dft=nbh.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
dft.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
#Check the number of rows of dataframe dft
dft.shape

(102, 3)